In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [16]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
N = len(y) # training set cardinality
D = tX.shape[1] # number of parameters ("dimensionality")

## Models 

Application of the 6 models expected for the project.

## Linear Regression using Gradient Descent


In [42]:
max_iters = 10
gamma = 0.01
initial_w = np.zeros([D,])
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)

In [43]:
print(w)
print(loss)

[-2.07818904e+42  1.04556522e+42  1.57564449e+42  6.06514350e+41
 -1.95732506e+43 -1.98860758e+43 -1.95702850e+43  4.89947586e+40
  2.80901486e+41  1.91845936e+42  2.70924522e+40 -9.63482351e+39
 -1.95719831e+43  6.80668247e+41 -3.12244815e+38 -2.43322917e+38
  8.41330528e+41 -5.84622132e+38  8.35473506e+38  6.57168494e+41
 -3.41859966e+38  2.89622491e+42  6.52585397e+39 -1.17036155e+43
 -1.21346345e+43 -1.21347402e+43 -1.95989842e+43 -1.95717167e+43
 -1.95717206e+43  3.96460581e+41]
9.027742767130377e+93


## Linear Regression usign Stochastic Gradient Descent

In [44]:
max_iters = 10
gamma = 0.01
batch_size = 1
initial_w = np.zeros([D,])
w, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma, batch_size)

In [45]:
print(w)
print(loss)

[-1.37041814e+41  9.92597555e+39  7.44311425e+39  3.09076918e+37
 -1.37039513e+41 -1.37039513e+41 -1.37039513e+41  2.74077109e+38
  3.09581207e+37  6.92363515e+39  2.04393060e+38 -1.90268993e+38
 -1.37039513e+41  2.78095260e+39 -3.26072412e+38  9.87687210e+37
  4.14295767e+39 -8.50515325e+37  2.29231239e+38  6.15708617e+39
 -2.52951967e+38  4.69015535e+39 -2.01893030e+33 -1.37041668e+41
 -1.37041537e+41 -1.37041532e+41 -1.37039513e+41 -1.37039513e+41
 -1.37039513e+41 -1.37944504e+35]
5.322675954542171e+89


## Least Squares Regression using Normal Equations

In [34]:
w, loss = least_squares(y, tX)

In [35]:
print(w)
print(loss)

[ 8.03494312e-05 -7.20202273e-03 -6.05417273e-03 -5.47559065e-04
 -1.93874700e-02  4.73451621e-04 -2.60379054e-02  3.25106300e-01
 -3.80780282e-05 -2.72724919e+00 -2.21220140e-01  9.50794091e-02
  6.40351613e-02  2.73550887e+00 -3.31801241e-04 -9.54325120e-04
  2.74026561e+00 -5.34164891e-04  9.73498581e-04  3.69225052e-03
  3.54487449e-04 -5.43344598e-04 -3.30448035e-01 -1.40800498e-03
  8.31432888e-04  1.02117272e-03 -1.68047416e-03 -5.83664818e-03
 -1.11087997e-02  2.72770912e+00]
0.3396868094770935


## Ridge Regression using Normal Equations
We perform a cross-validation for chosing the optimal lambda, degree is fixed to 5. # Tuning also for the degree?

In [48]:
# Should we put the cross-validation template in a separate, external file?

def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train:
    ind = k_indices[k,:]
    ind_tr = np.delete(k_indices, (k), axis = 0)
    x_tr = np.hstack(x[ind_tr])
    y_tr = np.hstack(y[ind_tr])
    # ridge regression:
    w = ridge_regression(y_tr, x_tr, lambda_)
    # calculate the loss for train and test data:
    e_tr = y_tr - basis_tr.dot(w)
    loss = 1/(2*len(y_tr)) * np.transpose(e_tr).dot(e_tr)
    
    return w, loss

seed = 1
degree = 5
k_fold = 3
lambdas = np.logspace(-4, 0, 30)
# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)
for i in range(len(lambdas)):
        lambda_ = lambdas[i]
        tr_loss = 0
        for k in range(k_fold): 
            loss_tr= cross_validation(y, tX, k_indices, k, lambda_, degree)[1]
            tr_loss = tr_loss + loss_tr
        rmse_tr.append(math.sqrt(2 * tr_loss/k_fold))


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 166666 is different from 83333)

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)